<a href="https://www.kaggle.com/fixfon/real-estate-price-changes-in-london?scriptVersionId=88418137" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Real Estate Price Changes in London
---
## Introduction

In this research, we will analyze how house prices changed in London over years. While analyzing prices, some questions will be asked and found out. In order to find accurate results, we will take some determinative paramaters in our data sets. Our data set will be combined with two data sets. One is *[Housing in London](https://www.kaggle.com/justinas/housing-in-london)* and the other one is *[1.6 million UK traffic accidents](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales)*. 

---
Before start to do some analysis about prices, we should examine our data sets and check if there is irrelevant, corrupted information in it.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

priceRawDF = pd.read_csv("../input/housing-in-london/housing_in_london_monthly_variables.csv")
housingYearRawDF = pd.read_csv("../input/housing-in-london/housing_in_london_yearly_variables.csv")
trafficRawDF = pd.read_csv("../input/2000-16-traffic-flow-england-scotland-wales/ukTrafficAADF.csv")

## Data Cleaning and Combining

In this section, we will anaylze our data sets columns and rows deeply to see if there is missing data and unusable data. Following, we will combine two data sets' some relevant columns each other to answer more question about house prices in London.

In [2]:
# Checking the empty cells. We will deal them in detail later.
print(priceRawDF.isnull().sum())
print(housingYearRawDF.isnull().sum())
print(trafficRawDF.isnull().sum())

date                0
area                0
average_price       0
code                0
houses_sold        94
no_of_crimes     6110
borough_flag        0
dtype: int64
code                   0
area                   0
date                   0
median_salary         22
life_satisfaction    719
mean_salary            0
recycling_pct        211
population_size       53
number_of_jobs       140
area_size            405
no_of_houses         405
borough_flag           0
dtype: int64
AADFYear                          0
CP                                0
Estimation_method             37573
Estimation_method_detailed    37573
Region                            0
LocalAuthority                    0
Road                              0
RoadCategory                      0
Easting                           0
Northing                          0
StartJunction                   256
EndJunction                     238
LinkLength_km                     0
LinkLength_miles                  0
PedalCycles     

In [3]:
citiesOfLondon = list(trafficRawDF[trafficRawDF['Region'] == 'London'].drop_duplicates(subset=['LocalAuthority'])['LocalAuthority']) # This list contains all the cities of London region of UK.
# ['city of london', 'city of westminster', 'kensington and chelsea', 'hammersmith and fulham', 'wandsworth', 'Lambeth', 'Southwark', 'Tower Hamlets', 'Hackney', 'Islington', 'Camden', 'Brent', 'Ealing', 'Hounslow', 'Richmond upon Thames', 'Kingston upon Thames', 'Merton', 'Sutton', 'Croydon', 'Bromley', 'Lewisham', 'Greenwich', 'Bexley', 'Havering', 'Barking and Dagenham', 'Redbridge', 'Newham', 'Waltham Forest', 'Haringey', 'Enfield' ,'Barnet', 'Harrow', 'Hillingdon']
citiesOfLondon = [x.lower() for x in citiesOfLondon]

# Extracting only London region and London cities of both two data sets.
housePricesDF = priceRawDF[priceRawDF['area'].str.lower().isin(citiesOfLondon)]
trafficCountsDF = trafficRawDF[(trafficRawDF['Region'] == 'London') & (trafficRawDF['LocalAuthority'].str.lower().isin(citiesOfLondon))]
housingYearRawDF = housingYearRawDF[housingYearRawDF['area'].str.lower().isin(citiesOfLondon)]

years = trafficCountsDF.drop_duplicates(subset=['AADFYear'])['AADFYear'] # Year interval of data set. (2000-2016)
# Extracting only the years that exist in both data sets.
# Before doing that we will summarize only year values from date in house price data set.

housePricesModified = housePricesDF.copy()
housePricesModified['date'] = pd.to_datetime(housePricesModified['date'], format='%Y-%m-%d')
housePricesModified['year'] = housePricesModified['date'].dt.year
housePricesModified.drop(columns=['date', 'no_of_crimes', 'borough_flag'], inplace=True) # Dropping unnecessary columns.

housingYearModified = housingYearRawDF.copy()
housingYearModified['date'] = pd.to_datetime(housingYearModified['date'], format='%Y-%m-%d')
housingYearModified['year'] = housingYearModified['date'].dt.year
housingYearModified.drop(columns=['date', 'life_satisfaction', 'recycling_pct', 'area_size', 'no_of_houses', 'borough_flag'], inplace=True) # Dropping unnecessary columns.

housePricesModified[(housePricesModified['year'] >= 2000) & (housePricesModified['year'] <= 2016)].isnull().sum() # Our selected data does not contain any null rows.
housingYearModified[(housingYearModified['year'] >= 2000) & (housingYearModified['year'] <= 2016)].isnull().sum() # Median column has some null values
housingYearModified[(housingYearModified['year'] >= 2000) & (housingYearModified['year'] <= 2016) & (housingYearModified['mean_salary'] == '#')] # We also have '#' replacement instead of null. We will replace it too.
housingYearModified['mean_salary'].replace('#', np.nan, inplace=True)
housingYearModified = housingYearModified.fillna({'mean_salary': 0})
housingYearModified = housingYearModified.astype({'mean_salary': 'int64'})

# Replacing empty rows with relevant average data.
for index, row in housingYearModified.iterrows():
    if(pd.isnull(row['median_salary'])):
        meanSalary = round(housingYearModified[housingYearModified['area'] == row['area']]['median_salary'].mean())
        housingYearModified.loc[index, 'median_salary'] = meanSalary
        
    if(row['mean_salary'] == 0):
        meanSalary2 = round(housingYearModified[housingYearModified['area'] == row['area']]['mean_salary'].mean())
        housingYearModified.loc[index, 'mean_salary'] = meanSalary2

In [4]:
# Combining two data sets with relevant columns and common years.
bigData = []
for year in years:
    for city in citiesOfLondon:
        housePricesSelected = housePricesModified[(housePricesModified['year'] == year) & (housePricesModified['area'] == city)]
        housingYearSelected = housingYearModified[(housingYearModified['year'] == year) & (housingYearModified['area'] == city)]
        trafficSelected = trafficCountsDF[(trafficCountsDF['AADFYear'] == year) & (trafficCountsDF['LocalAuthority'].str.lower() == city)]
        
        average_pr = round((housePricesSelected['average_price'].sum()) / 12)
        code = list(housingYearSelected['code'])[0]
        houses_sld = housePricesSelected['houses_sold'].sum()
        median_slry = list(housingYearSelected['median_salary'])[0]
        mean_slry = list(housingYearSelected['mean_salary'])[0]
        population_sz = list(housingYearSelected['population_size'])[0]
        number_of_jbs = list(housingYearSelected['number_of_jobs'])[0]
        vehicle_cnt = trafficSelected['AllMotorVehicles'].sum()
        dataArr = [year, city, average_pr, code, houses_sld, median_slry, mean_slry, population_sz, number_of_jbs, vehicle_cnt]
        bigData.append(dataArr)

combinedDF = pd.DataFrame(bigData, columns=['Year', 'City', 'Average_Price', 'Code', 'Houses_Sold', 'Median_Salary', 'Mean_Salary', 'Population_Size', 'Number_Of_Jobs', 'Vehicle_Count'])
combinedDF.head()

,Year,City,Average_Price,Code,Houses_Sold,Median_Salary,Mean_Salary,Population_Size,Number_Of_Jobs,Vehicle_Count
0,2000,barnet,167953,E09000003,6525.0,21761.0,25755,315784.0,138000.0,2845019
1,2000,hillingdon,131432,E09000017,5078.0,25038.0,29755,245911.0,177000.0,2326127
2,2000,tower hamlets,141288,E09000030,5026.0,28445.0,47253,197133.0,152000.0,2012116
3,2000,islington,192634,E09000019,2894.0,26598.0,35135,177852.0,173000.0,1473917
4,2000,southwark,142267,E09000028,4933.0,24857.0,29101,252726.0,182000.0,2364851


---
## In Which Cities Houses Have Been Sold Most?

London is one of nine regions of England and also known as *Greater London*. London region has 33 local government districts; the 32 London boroughs and the City of London, which is part of the region but forms a separate ceremonial county. (*[Greater London - Wikipedia](https://en.wikipedia.org/wiki/Greater_London)*) We will analyze London region's cities' house selling numbers over years.

In [5]:
citiesTitle = [city.title() for city in citiesOfLondon]
cityHouseCount = {}

for city in citiesTitle:
    cityHouseCount[city] = combinedDF[combinedDF['City'] == city.lower()]['Houses_Sold'].sum()

# Pie Chart
sortedCityHouseCount = dict(sorted(cityHouseCount.items(), key=lambda item: item[1], reverse=True))
cityLabel = list(sortedCityHouseCount.keys())[:16]
cityLabel.append('Others')

sellValues = list(sortedCityHouseCount.values())[:16]
sellValues.append(sum(list(sortedCityHouseCount.values())[16:]))

pieChart = go.Figure(data=[go.Pie(labels=cityLabel, values=sellValues, title="Number of Sold Houses in Cities of London (2000-2016)")])
pieChart.update_traces(hoverinfo='label+percent', textinfo='value')
pieChart.show()

---
### Insights
- 2,164,681 houses have been sold from 2000 to 2016.
- Inside of 2 million house sale, the city with the most house sale rate is Wandsworth with 112,450 houses.
- We will see in the following graphs that population, traffic flow rate, number of jobs etc. criterias like that have an impact on house sale numbers. 
---

## In Which Years Houses Have Been Sold Most?

Now we will look into cities' house sale numbers over years. To do so, we should first gather all years house sale numbers from all cities together to make comparison between years.

In [6]:
yearHouseCount = {}

for year in years:
    yearHouseCount[year] = combinedDF[combinedDF['Year'] == year]['Houses_Sold'].sum()
    
lineChart = go.Figure()
lineChart.add_trace(go.Scatter(
    x=list(yearHouseCount.keys()),
    y=list(yearHouseCount.values()),
    name = '',
    connectgaps=True 
))
lineChart.update_layout(title='Change on Number of House Sales in Years (2000-2016)',
                   xaxis_title='Year',
                   yaxis_title='House Sales')
lineChart.show()

---
### Insights
- While house sales are growing until 2008, it dropped suddenly due to global financial crisis of 2007 - 2008.
- The crisis affected nearly every business sector mainly in Europe as like house sales were affected.
- After 2008 when main crisis ended, house sales started to grow again.

---
## How Average House Prices Changed Over Years in City of London?

The major business and financial centre of UK, the historical city, City of London, has an important view in every research subject. We will look into its house prices over years and compare that with house sale numbers.

In [7]:
londonAPDict = {} # Average Prices of City Of London
wandsworthAPDict = {} # Average Prices of Wandsworth City

for year in years:
    londonAPDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'city of london')]['Average_Price'])[0]
    wandsworthAPDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'wandsworth')]['Average_Price'])[0]
    
lineChart2 = go.Figure()

lineChart2.add_trace(go.Scatter(
    x=years,
    y=list(londonAPDict.values()),
    name = '<b>City of London</b>',
    connectgaps=True
))
lineChart2.add_trace(go.Scatter(
    x=years,
    y=list(wandsworthAPDict.values()),
    name='<b>Wandsworth</b>',
))
lineChart2.update_layout(title='Average Price Comparison of City of London and Wansworth in Years',
                   xaxis_title='Year',
                   yaxis_title='Average Price of House')
lineChart2.show()

In [8]:
londonHSDict = {} # House sales of City Of London
wandsworthHSDict = {} # House sales of Wandsworth City

for year in years:
    londonHSDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'city of london')]['Houses_Sold'])[0]
    wandsworthHSDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'wandsworth')]['Houses_Sold'])[0]
    
lineChart3 = go.Figure()

lineChart3.add_trace(go.Scatter(
    x=years,
    y=list(londonHSDict.values()),
    name = '<b>City of London</b>',
    connectgaps=True
))
lineChart3.add_trace(go.Scatter(
    x=years,
    y=list(wandsworthHSDict.values()),
    name='<b>Wandsworth</b>',
))
lineChart3.update_layout(title='House Sale Comparison of City of London and Wansworth in Years',
                   xaxis_title='Year',
                   yaxis_title='House Sales')
lineChart3.show()

In [9]:
londonPopDict = {} # Population of City Of London
wandsworthPopDict = {} # Population of Wandsworth City

for year in years:
    londonPopDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'city of london')]['Population_Size'])[0]
    wandsworthPopDict[year] = list(combinedDF[(combinedDF['Year'] == year) & (combinedDF['City'] == 'wandsworth')]['Population_Size'])[0]
    
lineChart4 = go.Figure()

lineChart4.add_trace(go.Scatter(
    x=years,
    y=list(londonPopDict.values()),
    name = '<b>City of London</b>',
    connectgaps=True
))
lineChart4.add_trace(go.Scatter(
    x=years,
    y=list(wandsworthPopDict.values()),
    name='<b>Wandsworth</b>',
))
lineChart4.update_layout(title='Population Comparison of City of London and Wansworth in Years',
                   xaxis_title='Year',
                   yaxis_title='Population')
lineChart4.show()

---
### Insights
- We can answer many questions about house sales in these 3 line charts.
- We compared one of the biggest city in terms of house sales, Wandsworth, and one of the biggest city in terms of economic status.
- It is clear that average prices of houses in City of London is higher than Wandsworth, even house sales were low in City of London.
- Population is an important factor on house sales. The difference in house sales is caused mainly by population
> Short Info: How big is City of London?: 2.9km square.
>> How big is Central Park?: 3.41km square.
---
## How Salaries Affected House Sales?

Economic factors have significant impact on house sales and prices. Yearly salary averages should be examied as well.

In [10]:
meanSalaryDict = {}
houseSalesDict = {}

for year in years:
    meanSalaryDict[year] = round(combinedDF[combinedDF['Year'] == year]['Mean_Salary'].mean())
    houseSalesDict[year] = combinedDF[combinedDF['Year'] == year]['Houses_Sold'].sum()
    
lineChart5 = go.Figure()

lineChart5.add_trace(go.Scatter(
    x=years,
    y=list(meanSalaryDict.values()),
    name = '<b>Average Salary</b>',
    connectgaps=True
))
lineChart5.add_trace(go.Scatter(
    x=years,
    y=list(houseSalesDict.values()),
    name='<b>Houses Sold</b>',
))
lineChart5.update_layout(title='House Sales and Average Salaries (2000-2016)',
                   xaxis_title='Year',
                   yaxis_title='Count')
lineChart5.show()

---
### Insights
- Above chart shows that financial crisis affected house sales but average salaries in London still grew up over years.
- Average salaries stayed nearly same within years 2008 and 2009.
- After 2011, salaries dropped from 40k£ to 38k£ until 2015.
---
## Price Prediction for the Next Years

In [11]:
# Importing machine learning techniques and necesarry libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

modifiedData = []
for year in years: # Regional data creating.
    av_pr = round(combinedDF[combinedDF['Year'] == year]['Average_Price'].mean())
    hs = combinedDF[combinedDF['Year'] == year]['Houses_Sold'].sum()
    ms = round(combinedDF[combinedDF['Year'] == year]['Median_Salary'].mean())
    mes = round(combinedDF[combinedDF['Year'] == year]['Mean_Salary'].mean())
    popsize = combinedDF[combinedDF['Year'] == year]['Population_Size'].sum()
    noj = combinedDF[combinedDF['Year'] == year]['Number_Of_Jobs'].sum()
    vc = combinedDF[combinedDF['Year'] == year]['Vehicle_Count'].sum()
    dataArr = [year, av_pr, hs, ms, mes, popsize, noj, vc]
    modifiedData.append(dataArr)

londonRegionDF = pd.DataFrame(modifiedData, columns=['Year', 'Average_Price', 'Houses_Sold', 'Median_Salary', 'Mean_Salary', 'Population_Size', 'Number_Of_Jobs', 'Vehicle_Count'])

trainTestDF = londonRegionDF[~(londonRegionDF['Year'] == 2016)].copy() # 2000-2015 data will be used to train and test
predictDF = londonRegionDF[londonRegionDF['Year'] == 2016].copy() # 2016 data will be used to predict the prices.
predictDF.drop(columns=['Average_Price'], inplace=True)

features = trainTestDF[['Year', 'Houses_Sold', 'Median_Salary', 'Mean_Salary', 'Population_Size', 'Number_Of_Jobs', 'Vehicle_Count']] # X
target = trainTestDF['Average_Price'] # Y

train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.25, random_state=1)

models = [LinearRegression(),
          GradientBoostingRegressor(),
          RandomForestRegressor(),
          DecisionTreeRegressor(),
          KNeighborsRegressor(),
          AdaBoostRegressor()]

modelScores = {}

for model in models:
    my_pipeline = Pipeline(steps=[
                                  ('model', model)
                                 ])
    my_pipeline.fit(train_X, train_y)
    prediction = my_pipeline.predict(test_X)
    print(model, 'MAE:', mean_absolute_error(test_y, prediction), 'R2 Score:',r2_score(test_y, prediction), 'Model Fit Score:',my_pipeline.score(train_X, train_y))
    modelScores[type(model).__name__] = r2_score(test_y, prediction)

modelBar = go.Figure([go.Bar(x=list(modelScores.keys()), y=list(modelScores.values()))])
modelBar.update_layout(title='R2 Score of Some Alghoritms',
                   xaxis_title='Name',
                   yaxis_title='R2 Score')
modelBar.show()

# GradientBoostingRegressor will be used in our data even model fit score of DecisionTree is higher than this. 
#MAE and r2 score more reliable in GradientBossting alghoritm.

LinearRegression() MAE: 21020.79699351266 R2 Score: 0.8781173389219031 Model Fit Score: 0.9906660607490971
GradientBoostingRegressor() MAE: 17534.56022150453 R2 Score: 0.8972467192451501 Model Fit Score: 0.9999999990791947
RandomForestRegressor() MAE: 13918.019999999997 R2 Score: 0.9158017627066815 Model Fit Score: 0.9760915278935312
DecisionTreeRegressor() MAE: 38523.75 R2 Score: 0.6402331040855564 Model Fit Score: 1.0
KNeighborsRegressor() MAE: 22527.499999999993 R2 Score: 0.7919064989756786 Model Fit Score: 0.7120766345273182
AdaBoostRegressor() MAE: 30123.75 R2 Score: 0.8021832697778157 Model Fit Score: 0.995166889550785


In [12]:
# Test Validation Graph

testModel = GradientBoostingRegressor()
testModel.fit(train_X, train_y)
predicted = testModel.predict(test_X)

selectedIndexes = test_X.index.to_numpy()
selectedYears = np.take(years.to_numpy(), selectedIndexes)
selectedDict = {}
testDict = {}

for i in range(selectedYears.size):
    selectedDict[round(predicted[i])] = selectedYears[i]
    testDict[list(test_y)[i]] = selectedYears[i]
    
selectedDict = dict(sorted(selectedDict.items(), key=lambda item: item[1], reverse=False))
testDict = dict(sorted(testDict.items(), key=lambda item: item[1], reverse=False))

lineChart6 = go.Figure()
lineChart6.add_trace(go.Scatter(
    x=list(selectedDict.values()),
    y=list(selectedDict.keys()),
    name = '<b>Predicted Price</b>',
    connectgaps=True
))
lineChart6.add_trace(go.Scatter(
    x=list(selectedDict.values()),
    y=list(testDict.keys()),
    name='<b>Actual Price</b>',
))
lineChart6.update_layout(title='Tested House Prices in London (2000-2015)',
                   xaxis_title='Year',
                   yaxis_title='Average House Price')
lineChart6.show()

In [13]:
# House Price Prediction for 2016

forecast = testModel.predict(predictDF)

chartX = trainTestDF['Year'].to_numpy()
chartX = np.append(chartX, predictDF['Year'].to_numpy())
chartY = trainTestDF['Average_Price'].to_numpy()
chartY = np.append(chartY, forecast)

lineChart7 = go.Figure()
lineChart7.add_trace(go.Scatter(
    x=chartX,
    y=chartY,
    name = '<b>House Prices</b>',
    connectgaps=True
))
lineChart7.update_layout(title='2016 Predicted House Prices in London',
                   xaxis_title='Year',
                   yaxis_title='Average House Price')

lineChart7.add_shape( # Shape representing the surge
    type="rect",
    xref="x",
    yref="paper",
    x0="2015.5",
    y0=0,
    x1="2016.5",
    y1=1,
    fillcolor="lightgreen",
    opacity=0.5,
    layer="below",
    line_width=0
)

lineChart7.add_annotation(x=2016, y=350000,
            text="<b>Predicted Price</b>",
            showarrow=False)
lineChart7.show()

---
### Insights
- As we can see above, house prices in London region will be approximately 471k£ in 2016.
- Drop on house prices can be expected in 2016 than the previous year.
---

## Conclusion
To sum up, house prices in England, and especially in London region changed due to various factor over years. We analyzed some of the determining factor such as traffic flow in that city roads, population, house sales, number of jobs and salaries. After that, we predicted the new year's house prices in London using some machine learning techniques. At the end, we gathered the all information for an aspect of view to understand relevant factors on house prices over years. For further research subjects, more techniques can be implemented to predict more accurate results and some deep learning softwares may help to gather and process the data for the price prediction.